# BERT Model for Disaster Tweet Classification

In this Notebook, we will train a BERT model to classify tweets as disaster-related or not. We will use the preprocessed `train_data_mod.csv`, which contains the text and numerical features, to train the BERT model. <br>
We will be using `text + keyword + tweet length + punctuation count` as a feature. 

## Import the necessary Libraries

In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification, TrainingArguments, Trainer, TextClassificationPipeline
import re

## Load the Dataset

In [5]:
# Load dataset
train = pd.read_csv('../preprocessing/train_data_mod.csv')
test = pd.read_csv('../preprocessing/test_data_mod.csv')

In [6]:
# drop unnecessary columns
drop_cols = ['location', 'preprocess_text','bigram','trigram', 'pos']

In [7]:
train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)

In [8]:
# Concatenate keyword and text so that it can be passed to GPT-2 model as input
def concatenate_columns(df, text_col, keyword_col, tweet_length_col, punctuation_count_col):
    df['concatenated'] = df[text_col] + ' (Keyword: ' + df[keyword_col] + ', Tweet Length: ' + df[tweet_length_col].astype(str) + ', Punctuation Count: ' + df[punctuation_count_col].astype(str) + ')'
    return df

concatenate_columns(train, 'text', 'keyword', 'tweet_length', 'punctuation_count')
concatenate_columns(test, 'text', 'keyword', 'tweet_length', 'punctuation_count')


,id,keyword,text,keyword_encoded,tweet_length,punctuation_count,concatenated
0,0,missing,Just happened a terrible car crash,0.684304,34,0,Just happened a terrible car crash (Keyword: m...
1,2,missing,"Heard about #earthquake is different cities, s...",0.684304,64,3,"Heard about #earthquake is different cities, s..."
2,3,missing,"there is a forest fire at spot pond, geese are...",0.684304,96,2,"there is a forest fire at spot pond, geese are..."
3,9,missing,Apocalypse lighting. #Spokane #wildfires,0.684304,40,3,Apocalypse lighting. #Spokane #wildfires (Keyw...
4,11,missing,Typhoon Soudelor kills 28 in China and Taiwan,0.684304,45,0,Typhoon Soudelor kills 28 in China and Taiwan ...
...,...,...,...,...,...,...,...
3258,10861,missing,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,0.684304,55,0,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,missing,Storm in RI worse than last hurricane. My city...,0.684304,139,5,Storm in RI worse than last hurricane. My city...
3260,10868,missing,Green Line derailment in Chicago http://t.co/U...,0.684304,55,5,Green Line derailment in Chicago http://t.co/U...
3261,10874,missing,MEG issues Hazardous Weather Outlook (HWO) htt...,0.684304,65,7,MEG issues Hazardous Weather Outlook (HWO) htt...


In [9]:
train_data = train.copy()
test_data = test.copy()

#### To use `concatenated` Feature

In [11]:
# Split the training data into train set and validation set
X_train, X_val, y_train, y_val = train_test_split(train_data["concatenated"], train_data["target"], test_size=0.2, random_state=42)

## Load and Prepare BERT Model

#### Dataset for `concatenated` feature

In [12]:
# Load the BERT tokenizer, model, and configuration
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
config = BertConfig.from_pretrained("bert-base-uncased", num_labels=2)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)

# Tokenize the training and validation data
train_encodings = tokenizer(X_train.to_list(), truncation=True, padding=True)
val_encodings = tokenizer(X_val.to_list(), truncation=True, padding=True)

# Create a dataset object for the trainer
class DisasterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = DisasterDataset(train_encodings, y_train.to_list())
val_dataset = DisasterDataset(val_encodings, y_val.to_list())


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Train the BERT Model (`concatenated` feature)

In [24]:
# Set up the training arguments and trainer
training_args = TrainingArguments(
    output_dir="./BERT_results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=100,
    evaluation_strategy="steps",
    save_strategy="no",
    learning_rate=1e-3,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

c:\Users\yuhao\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2286 [00:00<?, ?it/s]

{'loss': 0.6148, 'learning_rate': 4.781277340332459e-05, 'epoch': 0.13}


  0%|          | 0/191 [00:00<?, ?it/s]

{'eval_loss': 0.4875624179840088, 'eval_runtime': 255.2915, 'eval_samples_per_second': 5.966, 'eval_steps_per_second': 0.748, 'epoch': 0.13}
{'loss': 0.482, 'learning_rate': 4.562554680664917e-05, 'epoch': 0.26}


...

  0%|          | 0/191 [00:00<?, ?it/s]

{'eval_loss': 0.5548914074897766, 'eval_runtime': 266.4849, 'eval_samples_per_second': 5.715, 'eval_steps_per_second': 0.717, 'epoch': 2.76}
{'loss': 0.3882, 'learning_rate': 1.8810148731408575e-06, 'epoch': 2.89}


  0%|          | 0/191 [00:00<?, ?it/s]

{'eval_loss': 0.5187697410583496, 'eval_runtime': 261.3173, 'eval_samples_per_second': 5.828, 'eval_steps_per_second': 0.731, 'epoch': 2.89}
{'train_runtime': 15537.9677, 'train_samples_per_second': 1.176, 'train_steps_per_second': 0.147, 'train_loss': 0.42315439456195447, 'epoch': 3.0}


TrainOutput(global_step=2286, training_loss=0.42315439456195447, metrics={'train_runtime': 15537.9677, 'train_samples_per_second': 1.176, 'train_steps_per_second': 0.147, 'train_loss': 0.42315439456195447, 'epoch': 3.0})

In [ ]:
# save trainer (concatenated)
trainer.save_model('./BERT_saved_model_concatenated')

In [14]:
# load trainer (concatenated)
model = BertForSequenceClassification.from_pretrained('./BERT_saved_model_concatenated')

## Make predictions on Test Data

In [25]:
# Make predictions on the test data
test_pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer)
predictions = test_pipeline(test_data["concatenated"].to_list())

# Save the predictions to a CSV file
test_data["target"] = [prediction["label"].split("_")[-1] for prediction in predictions]
test_data[["id", "target"]].to_csv("BERT_predictions/BERT_predictions_concatenated.csv", index=False)

## Conclusion

This achieved an accuracy score of Score: `0.81673` based on the Kaggle submission page. <br>
 This is significantly higher than the scores achieved by the traditional models we have trained. <br>
 It is important to note that the model training took more than `10 hours`, so there is a need to optimize the training speed such as the approach taken by `BERT_CC.ipynb` and `BERT_numerical.ipynb` by getting the hidden states instead. Due to the lack of time, this is a sufficiently good enough model without further hyperparameter fine-tuning.